# Landscape generation

In this notebook we show how to use TopSearch to generate the landscape of the two-dimensional Schwefel function. We first locate the local minima of this surface using the basin-hopping algorithm, and subsequently locate the transition states between them. This set of minima and transition states can be represented as a weighted graph, and we show how this graph encodes the topography of the surface.

## Imports

First we import topsearch, which contains all the functionality we need for this example

In [ ]:
import topsearch

## Initialise classes

Landscape generation requires several classes to be instantiated. We list each of them in turn here, and describe their functionality.
First, we generate the potential class that calculates the function value and its derivatives when given a set of coordinates. In this case this is the Schwefel function specified in two dimensions. We can place bounds on the allowed feature space that is explored by these methods, here they are given the standard range for the Schwefel function.

In [ ]:
schwefel = topsearch.potential.Schwefel(ndim=2, bounds=[(-500.0, 500.0), (-500.0, 500.0)])

Next we initialise the comparison class, which is responsible for determining the distance between points in feature space and determining if they are the same. We may generate repeated minima and transition states and the functions of this class will allow us to filter out repeats and only include unique stationary points. We are required to specify a distance_criterion and energy_criterion within which two minima and transition states are considered the same. The proportional_distance scales the distance_criterion to reflect the total range, in this case it is active resulting in the criterion taking any points within 5% of the feature space and within the energy_criterion as the same.

In [ ]:
comparer = topsearch.similarity.NonAtomicSimilarity(potential=schwefel,
                                                    distance_criterion=0.05,
                                                    energy_criterion=1e-2,
                                                    proportional_distance=True)

We provide these two instances to the KineticTransitionNetwork class. This contains the networkx object in which all the minima and transition states are stored as a weighted graph. This class controls all access to the network and performs operations to extract information and analyse it.

In [ ]:
ktn = topsearch.kinetic_transition_network.KineticTransitionNetwork(potential=schwefel,
                                                                    similarity=comparer)

Additionally, we need a local minimiser as all these methods rely on locating local minima. Our local minimiser class provides a wrapper to the scipy box-constrained LBFGS implementation that is adapted to the different optimisation tasks we perform. We need to provide it the function that it minimises, along with the maximum number of steps it can take, the LBFGS history and gradient at which it has successfully converged.

In [ ]:
minimiser = topsearch.minimisation.LBFGS(potential=schwefel,
                                         conv_crit=1e-6,
                                         history_size=5,
                                         n_steps=100)

Locating transition states requires a combination of single-ended and double-ended searches. Double-ended searches take two minima as input and attempt to find the lowest-valued path between them. Single-ended searches start from a single point and follow the eigenvector corresponding to the most negative eigenvalue until they converge to a transition state. These two searches are used in tandem, with an initial double-ended search, following by a single-ended search applied to each local maximum on the path. Here, we use the nudged elastic band as the double-ended search method. NudgedElasticBand contains the methods to produce an initial path and optimise it to minimise its overall value. We specify it using three parameters: force_constant - determines the tightness of the elastic band, this is updated within the computation, image density - this determines how many points the path is composed of (per unit distance), more points means better path (usually) as higher computational cost, max images - allows us to put a cut on the maximum number of images to limit the computational cost.

In [ ]:
nudged_elastic_band = \
    topsearch.double_ended_search.NudgedElasticBand(potential=schwefel,
                                                    minimiser=minimiser,
                                                    force_constant=50.0,
                                                    image_density=5.0,
                                                    max_images=50)

The single-ended search method is hybrid eigenvector-following. This class provides methods to take in a single point and provide the methods to converge it to the nearest transition state. We provide it with a convergence criterion, the gradient must be under this value to be considered converged to a stationary point. We also provide the allowed number of mode-following steps before it is considered a failed search for a transition state. Each transition state is connected to two minima by following the steepest-descent paths along the unique downhill direction and we specify the distance we move in the downhill direction before beginning a local minimisation using pushoff. Finally, we have some step sizes used in the mode-following to prevent the steps being too large.

In [ ]:
hybrid_eigenvector_following = \
    topsearch.single_ended_search.HybridEigenvectorFollowing(potential=schwefel,
                                                             minimiser=minimiser,
                                                             conv_crit=1e-4,
                                                             ts_steps=100,
                                                             pushoff=1e-1,
                                                             max_uphill_step_size=0.5,
                                                             positive_eigenvalue_step=1.0)

For global optimisation algorithms we need to be able to propose new structures from the existing ones. The efficiency of global optimisation relies on the proposal of good candidate positions. For molecular systems this is an involved problem with a lot of research invested into it. For the non-atomic system here the step-taking is much simpler, it can just be random perturbations. This class manages the perturbations to propose new states, it is given the maximum step size, max_displacement, and we specify that this distance should be measured as a proportion of the bounds range with proportional distance.

In [ ]:
step_taking = topsearch.perturbations.NonAtomicPerturbation(potential=schwefel,
                                                            max_displacement=1.0,
                                                            proportional_distance=True)

Initialise the global optimisation class. The global optimisation algorithm is basin-hopping, which is provided with the step-taking class previously created. Basin-hopping steps around the surface performing local minimisations and subsequently accepting or rejecting the new local minima based on a Metropolis-like criterion. The BasinHopping class performs basin-hopping runs consisting of n_steps random perturbations and local minimisations, with a temperature specified to control the acceptance of new minima.

In [ ]:
optimiser = topsearch.global_optimisation.BasinHopping(ktn=ktn,
                                                       minimiser=minimiser,
                                                       n_steps=750,
                                                       temperature=1.0,
                                                       step_taking=step_taking)

Finally, we feed many of these objects into a NetworkSampling object that controls all the landscape generation. This object allows for simple calls to be made that perform the combination of algorithms for landscape generation. We pass it the global_optimiser and the transition state location methods. Transition state location simply requires two minima, and therefore, the sampling of the landscape is embarrassingly parallel. We have the option to use multiple processes to accelerate the landscape calculation. Here we decide to run on one CPU for now.

In [ ]:
explorer = topsearch.exploration.NetworkSampling(ktn=ktn,
                                                 minimiser=minimiser,
                                                 global_optimiser=optimiser,
                                                 single_ended_search=hybrid_eigenvector_following,
                                                 double_ended_search=nudged_elastic_band,
                                                 multiprocessing=False,
                                                 n_processes=1)

## Perform the computations

We will perform several tasks in turn for the Schwefel function, each illustrating tasks which may be performed separately
1. Global optimisation
2. Find transition states
3. Restart from partial landscape
4. Restart with multiprocessing

## 1. Global optimisation

First, we will locate minima on the two-dimensional Schwefel function surface. We will use the basin-hopping algorithm, as specified in the optimiser class. Basin-hopping is a very efficient global optimisation algorithm, and we store all additional minima that are located during the search too. To perform global optimisation now requires a single line of code

In [ ]:
explorer.global_optimisation()

We can analyse and visualise these results to ensure that the methodology is correctly exploring the space and locating minima. First, because we selected a two-dimensional surface we can directly visualise the surface to validate its performance. We do this through a call to the plotter class.

In [ ]:
plotter.plot_stationary_points(ktn)

## 2. Find transition states

Having successfully located the local minima we will aim to locate the transition states between them. These transition states are the maxima on the minimum energy paths between minima, and in physical landscapes these are the lowest crossing points between two valleys. Hopefully, the concept will be straightforward to see when we plot them shortly.

The transition states allow us to understand the intermediate structure of the surface between minima through these key points. We can then encode the topography through these key points and represent the structure of the surface as a weighted graph. Generation of the transition states is performed by using the combination of single and double-ended transition state searches between minima.

The algorithm we use 'ClosestEnumeration' specifies that all minima are connected to their nearest neighbour with transition state searches. Again, the generation of the complete landscape description can be performed in a single function call.

In [ ]:
explorer.generate_landscape('ClosestEnumeration', 1)

We have only taken a small number of connections here, and we visualise the parts of the landscape we have captured.

In [ ]:
plotter.plot_stationary_points(ktn)

It is easy to see that we have not found all transition states between the minima. Therefore, we currently have an incomplete description of the surface topography. Additional transition state searches are needed to explore more of the surface. Assuming that at this point the calculations want to be paused we will dump the current network of minima and transition states to file for storage. These files are min.data, min.coords, ts.data, ts.coords and pairlist.txt.

In [ ]:
ktn.dump_network()

## 3. Restarting calculations

Here, we will show how it is possible to restart calculations from an existing landscape. We previously did a small amount of transition state searching, but it was not enough to explore the full surface. Therefore, we would like to continue sampling from this database. Reading in an existing database is performed using

In [ ]:
ktn.read_network()

which will take in all the network information written to file and reconstruct the graph for future calculations. We can then continue transition state searches to further explore the surface from where we left off. The same method can be called again to continue from where we left off, but this time we try and connect each minimum with its three nearest neighbours.

In [ ]:
serial_start_time = timer()
explorer.generate_landscape('ClosestEnumeration', 3)
serial_end_time = timer()

Now we have further explored the surface we will again visualise its progress. We visualise the surface directly with the graph overlaid. This visualisation is only possible due to the low-dimensionality of the problem. Therefore, we show two different visualisations that can capture the topography for surfaces of any dimensionality: disconnectivity graphs and graphs. First, we show the surface directly

In [ ]:
plotter.plot_stationary_points(ktn)

We now observe that far more of the surface is covered by the network. Therefore, the network provides a much better description of the surface topography. Highlighting almost all the of the key points in moving between minima. However, such a visualisation is only possible two dimensions and we illustrate two visualisations that are possible in arbitrary dimensions.

In [ ]:
plotter.plot_disconnectivity_graph(ktn)
plotter.plot_network(ktn)

The network plot illustrates the graph in a representation that respects the barriers between minima to enforce edges to be shorter if the transition state barrier is lower. Therefore, the graph gives a visual illustration of surface topography in two dimensions, highlighting groups of nodes that are close together as those within a smooth part of the surface. The disconnectivity graph is another two-dimensional representation of surface topography that preserves the true barriers between minima.

## 4. Restart with multiprocessing

The transition state searches are embarassingly parallel, as each only needs to know the initial two minima that it is connecting. Therefore, we can use multiprocessing to further accelerate the exploration of the surface by running each separate transition state search on a different process. After an initial generation of the set of minima that should be connected we spread the calculations across n_processes. Usually we would specify this option in NetworkSampling, but here we simply update these values

In [ ]:
explorer.multiprocessing = True
explorer.n_processes = 4

Now any landscape exploration will be performed in parallel over n_processes different processes. We read in the network in the same state as for the previous serial computation and perform the same set of calculations, this time with multiple processes.

In [ ]:
ktn.read_network()
parallel_start_time = timer()
explorer.generate_landscape('ClosestEnumeration',3)
parallel_end_time = timer()
print("Serial computation time = ", serial_end_time - serial_start_time)
print("Parallel computation time = ", parallel_end_time - parallel_start_time)

Printing the timings for both of these methods we can show that the multiprocessing provides a large reduction in computational time when multiple cores are available.